In [28]:
import networkx as nx
import os
import time
import random
import pandas as pd
import numpy as np
from networkx.algorithms.community import kernighan_lin_bisection

In [44]:
def bfs(visited, queue, graph, node):
    visited.append(node)
    queue.append(node)
    count = 1

    while queue:
        m = queue.pop(0) 

        for neighbour in graph.neighbors(m):
            if neighbour not in visited:
                count = count + 1
                visited.append(neighbour)
                queue.append(neighbour)
    return count

def bfs_count(G):
    visited = []
    queue = []

    return bfs(visited, queue, G, np.asarray(G.nodes)[0])

def binary_search(arr, x):
  low = 0
  high = len(arr) - 1
  mid = 0

  while low <= high:

    mid = (high + low) // 2

    if arr[mid] < x:
      low = mid + 1
    elif arr[mid] > x:
      high = mid - 1
    else:
      return mid
  return -1


def find_edges(graph, p1, p2):

  edge_list = np.array(graph.edges)
  node_list = np.array(graph.nodes)

  g1 = nx.Graph()
  g2 = nx.Graph()

  color_map = np.empty(len(node_list), dtype=np.str_)

  for edge in edge_list:
    if (edge[0] in p1) and (edge[1] in p1):
      g1.add_edge(edge[0], edge[1])

  for edge in edge_list:
    if (edge[0] in p2) and (edge[1] in p2):
      g2.add_edge(edge[0], edge[1])
  
  for node in p1:
    color_map[int(node)] = 'yellow'

  for node in p2:
    color_map[int(node)] = 'red'

  return (g1, g2, color_map)

def partition(G, partition=None, max_iter=10):
  blocks = kernighan_lin_bisection(G, partition=partition, max_iter=max_iter)
  p1 = np.array(list(blocks[0]))
  p2 = np.array(list(blocks[1]))

  return find_edges(G, p1, p2)

def isValid(g1, g2):
  c1 = bfs_count(g1)
  c2 = bfs_count(g2)

  if c1 != len(g1.nodes) or c2 != len(g2.nodes):
    return False
  return True

def labelize(G, df):
  g1, g2, color_map = partition(G)
  label = isValid(g1, g2)
  df.loc[len(df.index)] = [np.asarray(G.edges), label]
  return df


In [45]:
random.seed(time.time() % 60)

def generate_graphs(model, num_graphs, graph_size, df):
    #m, k, p ve d değişkenler; bunları değiştirerek üretin
    #m=Number of edges to attach from a new node to existing nodes
    #n=Number of nodes in graph
    #k=Each node is connected to k nearest neighbors
    #p=Probability of edge existence
    #d=Degree of each node
    for i in range(num_graphs):
        probability = random.random() / 10 + 0.2
        k = random.randint(1, 5)
        # Generate a random graph based on the specified model
        if model == "erdos_renyi":
            G = nx.erdos_renyi_graph(graph_size, probability)
        elif model == "barabasi_albert":
            m = 2  # Define m before using it
            G = nx.barabasi_albert_graph(graph_size, m)
        elif model == "watts_strogatz":
            k = 2  # Define k before using it
            G = nx.watts_strogatz_graph(graph_size, k, probability)
        elif model == "random_regular":
            d = 4  # Define d before using it
            G = nx.random_regular_graph(d, graph_size)
        elif model == "random_geometric":
            G = nx.random_geometric_graph(graph_size, probability)
            
        df = labelize(G)
    
    return df


# Parameters
num_graphs_per_model = 1
#bu da değişebilir her modelin graph sayısı
n = 1000

dataset = []

df = pd.DataFrame()

# Generate and save graphs for each model
models = ["erdos_renyi", "barabasi_albert", "watts_strogatz", "random_regular", "random_geometric"]
for model in models:
    dataset = generate_graphs(model, num_graphs_per_model, n, df)

file_name = 'graph_data_' + str(n)


df.to_csv(file_name, index=False)
